In [50]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE

In [2]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\project-III-geospatial-data'

In [3]:
# aqui estamos haciendo un query a una pagina we que nos da
# el punto georeferencia de la ciudad que vuscamos.
# lo sometimos a una funcion de formato. 

In [4]:
where = "Barcelona, Spain"

In [5]:
res = requests.get(f"https://geocode.xyz/{where}?json=1").json()
res

{'standard': {'addresst': {},
  'statename': {},
  'city': 'Barcelona',
  'prov': 'ES',
  'countryname': 'Spain',
  'postal': {},
  'confidence': '0.90'},
 'longt': '2.15379',
 'alt': {'loc': {'longt': '2.19306',
   'prov': 'ES',
   'city': 'Barcelona',
   'countryname': 'Spain',
   'postal': '08030',
   'region': {},
   'latt': '41.43553'}},
 'elevation': {},
 'latt': '41.40359'}

In [6]:
res

{'standard': {'addresst': {},
  'statename': {},
  'city': 'Barcelona',
  'prov': 'ES',
  'countryname': 'Spain',
  'postal': {},
  'confidence': '0.90'},
 'longt': '2.15379',
 'alt': {'loc': {'longt': '2.19306',
   'prov': 'ES',
   'city': 'Barcelona',
   'countryname': 'Spain',
   'postal': '08030',
   'region': {},
   'latt': '41.43553'}},
 'elevation': {},
 'latt': '41.40359'}

In [7]:
res["longt"]

'2.15379'

In [8]:
res["latt"]

'41.40359'

In [9]:
# esta es la ubicacion que estamos unsando y la podemos 
# sustituir en el link API 
# al momento de hacerlo una funcion. 

In [10]:
mad_long = 2.15379
mad_lat = 41.40359

In [11]:
load_dotenv()

True

In [12]:
# estas son las credenciales del API,
# la web nos dio el chance de ponerlos pero podemos hacerlo en una 
# funcion para que sea mas versatil

In [13]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
client_key = os.getenv("API_KEY")

In [14]:
# aqui estamos haciendo un query directo al API de la web con los
# los parametros que le dimeos al foursquare

In [32]:
import requests

url = "https://api.foursquare.com/v3/places/search?query=coffe&ll=41.40359%2C2.15379&radius=10000"

headers = {
    "Accept": "application/json",
    "Authorization": client_key  #the variable has to be without ""
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"5315c94d11d208323008b630","categories":[{"id":13035,"name":"Coffee Shop","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_","suffix":".png"}},{"id":13343,"name":"South American Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/argentinian_","suffix":".png"}}],"chains":[],"distance":780,"geocodes":{"main":{"latitude":41.398159,"longitude":2.159823},"roof":{"latitude":41.398159,"longitude":2.159823}},"link":"/v3/places/5315c94d11d208323008b630","location":{"address":"Calle de Santa Teresa, 1","admin_region":"Cataluña","country":"ES","cross_street":"C. Bonavista","formatted_address":"Calle de Santa Teresa, 1 (C. Bonavista), 08012 Barcelona Catalonia","locality":"Barcelona","postcode":"08012","region":"Catalonia"},"name":"Onna Coffee","related_places":{},"timezone":"Europe/Madrid"},{"fsq_id":"4fda2fc2e4b02b4ce2b26007","categories":[{"id":13003,"name":"Bar","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightli

In [16]:
# Aqui estamos guardando el texto en un comando lejible para python
# una vez que sacamos esto podemos usar la informaci'on

In [17]:
test = response.json()

In [18]:
test

{'results': [{'fsq_id': '5315c94d11d208323008b630',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13343,
     'name': 'South American Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/argentinian_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 780,
   'geocodes': {'main': {'latitude': 41.398159, 'longitude': 2.159823},
    'roof': {'latitude': 41.398159, 'longitude': 2.159823}},
   'link': '/v3/places/5315c94d11d208323008b630',
   'location': {'address': 'Calle de Santa Teresa, 1',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': 'C. Bonavista',
    'formatted_address': 'Calle de Santa Teresa, 1 (C. Bonavista), 08012 Barcelona Catalonia',
    'locality': 'Barcelona',
    'postcode': '08012',
    'region': 'Catalonia'},
   'name': 'Onna Coffee',
   'related_places': {},
   'timezone': '

In [40]:
# we check on the keys to identify what is in the list we can see 2 keys
test.keys()

dict_keys(['results', 'context'])

In [35]:
# from our query we received a list of 10 dictionaries with locations that fit our search in foursquare.
len(test["results"])

10

In [42]:
test["context"]

{'geo_bounds': {'circle': {'center': {'latitude': 41.40359,
    'longitude': 2.15379},
   'radius': 10000}}}

In [19]:
#now lets access the information and deep dive to locate the information that can use.
test["results"][0] 

{'fsq_id': '5315c94d11d208323008b630',
 'categories': [{'id': 13035,
   'name': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'}},
  {'id': 13343,
   'name': 'South American Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/argentinian_',
    'suffix': '.png'}}],
 'chains': [],
 'distance': 780,
 'geocodes': {'main': {'latitude': 41.398159, 'longitude': 2.159823},
  'roof': {'latitude': 41.398159, 'longitude': 2.159823}},
 'link': '/v3/places/5315c94d11d208323008b630',
 'location': {'address': 'Calle de Santa Teresa, 1',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': 'C. Bonavista',
  'formatted_address': 'Calle de Santa Teresa, 1 (C. Bonavista), 08012 Barcelona Catalonia',
  'locality': 'Barcelona',
  'postcode': '08012',
  'region': 'Catalonia'},
 'name': 'Onna Coffee',
 'related_places': {},
 'timezone': 'Europe/Madrid'}

In [20]:
# following thte extraction of information of the API 
# we proceed to locate the information to create the DF we need

In [21]:
test["results"][0]["geocodes"]["main"]["latitude"]

41.398159

In [22]:
test["results"][0]["geocodes"]["main"]["longitude"]

2.159823

In [23]:
test["results"][0]["name"]

'Onna Coffee'

In [24]:
list_of_coffestore = test["results"]

In [25]:
new_list = []
for i in list_of_coffestore:
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    long = i["geocodes"]["main"]["longitude"]
    
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, long]}}
    
    new_list.append({"name":name, "lat":lat, "long":long, "type":type_})
new_list[0]

{'name': 'Onna Coffee',
 'lat': 41.398159,
 'long': 2.159823,
 'type': {'typepoint': {'type': 'Point',
   'coordinates': [41.398159, 2.159823]}}}

In [26]:
coffeshop = pd.DataFrame(new_list)
coffeshop

,name,lat,long,type
0,Onna Coffee,41.398159,2.159823,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Satan's Coffee Corner,41.382592,2.175354,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Nømad Coffee Lab & Shop,41.389072,2.176983,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Cafés el Magnífico,41.383628,2.181152,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Satan's Coffee,41.393575,2.174295,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Hidden Café Barcelona,41.388303,2.136004,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Roast Club Cafe,41.387250,2.155725,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,Syra Coffee,41.400954,2.159432,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,Federal,41.378062,2.177878,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,SlowMov,41.398040,2.154868,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [33]:
#only 10 locations where found on the location givern with a radious of 10 kilometers

len(coffeshop)

10

In [27]:
coffeshop2 = gpd.GeoDataFrame(coffeshop, geometry=gpd.points_from_xy(coffeshop["long"], coffeshop["lat"]))
coffeshop2.head()

,name,lat,long,type,geometry
0,Onna Coffee,41.398159,2.159823,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (2.15982 41.39816)
1,Satan's Coffee Corner,41.382592,2.175354,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (2.17535 41.38259)
2,Nømad Coffee Lab & Shop,41.389072,2.176983,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (2.17698 41.38907)
3,Cafés el Magnífico,41.383628,2.181152,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (2.18115 41.38363)
4,Satan's Coffee,41.393575,2.174295,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (2.17429 41.39357)


In [43]:
# we Use map from carto frame to project the locations to the geo reference we anted. 
Map(Layer(coffeshop2, "color:purple", popup_hover=[popup_element("name")]))

In [ ]:
# te venue we are looking for might have a space in the name, the API takes those 
# spaces fill them with "%20" in this case we will need to transform our input data
if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")

In [ ]:
# this will turn the integer of codes to string to be able to use in the web page
{str(location[0]).strip()}%2C{str(location[1]).strip()}

In [ ]:
# this final piece of the API web query is the radius of vecinity from point of reference up to 100,000 mtrs
# the limit is how we mant to mage the zise of the 
&radius={radius}&limit={limit}"

In [ ]:
# this creates the premise to create a list with all the names, lat, lon and type
# for every iteration in the loop, this can be later transformed to a dataframe
if response:
        response=response.json()['results']
        new_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, long]}}} for i in response]

In [ ]:
# this will give grab the list of dictionaries created and turn them into a dataframe
# after the dataframe will add a column be converted to geo points into usable coordinates for Map
# then we can project the information using Map and layer, to create 

df = pd.DataFrame(new_list)
    df_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["long"], df["lat"]))
    
    return Map(Layer(df_geo, "color:purple", popup_hover=[popup_element("name")]))
        

In [29]:
# if we put it all together it should work as intended, a 
# cascade of activities to reach a final output of a dataframe ready to work 

def query4sqr (query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&sort=DISTANCE&limit=50"
    headers =  {
    "Accept": "application/json",
    "Authorization": f"{client_key}" # tengo que validar si poner la llave en la funcion es factible.
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        new_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, long]}}} for i in response]
    
        df = pd.DataFrame(new_list)
        df_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["long"], df["lat"]))
        
        return Map(Layer(df_geo, "color:purple", popup_hover=[popup_element("name")]))
        
    else:
        return f"Sorry, no matches for {query} in the coordinates {location}"

In [ ]:
# this will give the individual points in the map for every run, it has a problem when 
# you want to analize several criterias, best option would be to use the same logic but
# with "FOLIUM" , it will allow for us to be able to create different markers for the 
# different type of locations for our desired analizis hot spot for our companies venue 

In [ ]:
# CHECK !!!! to see transparancy, radious of circle, overlaping of ranges. 

     #Create a circular marker for radious range with folium:
    "https://stackoverflow.com/questions/50954840/displaying-radius-in-meters-with-folium"
    
    # Carto frames visualization maps how to:
    "https://carto.com/developers/cartoframes/guides/Data-Visualization/"
    
# check count of places near us and what is a priority:
    # grooming(company dog)
    # daycare, school
    # designes(school, seminars, companies) 
    # **STARBUCKS**(Executives love it, high priority?)
    # CEO vegan optios! 
    # near successful startups (for developers, maybe not search this one)
    # basketball court (for maintinance guy)
    # Place to party (for 25 to 40 ys employees)
    # Airport (for account managers)
    
# check distance of every venue 

In [ ]:
     #BUT WHAT CITY? checked the web and Berlin has a good enviroment and i like the city:
      #"https://startupgenome.com/article/rankings-2021-top-30-plus-runners-up"
     
     #Check best areas in every city of interest for work.
        # companies similar to the areas that we have interest in>


In [ ]:
# same Formula in Folium:

In [ ]:
def query4sqr (query, location, radius, limit, key):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit={limit}"
    headers =  {
    "Accept": "application/json",
    "Authorization": f"{key}" # tengo que validar si poner la llave en la funcion es factible.
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        new_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, long]}}} for i in response]
        df = pd.DataFrame(new_list)
        
        for index, row in df.iterrows():
            
            icon = Icon (
            color="red",
            prefix="fa",
            icon=f"{query}",
            icon_color = "white")
            
            venues = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
            new_marker = Marker(**venues, icon=icon)
            new_marker.add_to(destination_map)

In [ ]:
for index, row in df.iterrows():
    
     # 1. Location (and some other things like the tooltip)
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["distrito"]}

    
    # 2. Icon based on the value
    if row["festividad"] == "Laborable": # work day
        icon = Icon (
            color="red",
            prefix="fa",
            icon="laptop",
            icon_color = "black")
         
    elif row["festividad"] == "Vispera": #eve day
            icon = Icon (
                color="blue",
                prefix="fa",
                icon="glass",
                icon_color = "black")
        
    elif row["festividad"] == "Festivo":
            icon = Icon (
                color="darkpurple",
                prefix="fa",
                icon="bug",
                icon_color = "black")
            
    else:
            icon = Icon (
                color="darkpurple",
                prefix="fa",
                icon="paypal",
                icon_color = "yellow")
        
    new_marker = Marker(**district, icon=icon)
    new_marker.add_to(madrid_map)
madrid_map

In [44]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\project-III-geospatial-data'

In [45]:
with open (".\\geopython\\data\\berlin-neighbourhoods.geojson") as berlin:
    berlin_districts = json.load(berlin)

In [46]:
berlin_districts

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[13.411909, 52.614871],
       [13.411826, 52.614838],
       [13.410914, 52.614604],
       [13.410822, 52.614578],
       [13.408516, 52.613961],
       [13.407365, 52.613653],
       [13.40751, 52.613593],
       [13.409923, 52.611614],
       [13.410344, 52.611269],
       [13.410476, 52.611161],
       [13.411191, 52.610709],
       [13.412582, 52.609868],
       [13.413236, 52.609557],
       [13.413555, 52.609405],
       [13.413611, 52.609223],
       [13.415022, 52.605169],
       [13.415149, 52.604873],
       [13.415831, 52.603062],
       [13.415854, 52.602967],
       [13.415843, 52.602537],
       [13.415996, 52.602017],
       [13.41626, 52.601544],
       [13.41659, 52.600657],
       [13.416776, 52.600064],
       [13.416871, 52.599751],
       [13.417183, 52.599564],
       [13.417927, 52.598957],
       [13.419275, 52.597546],
       [13.41967

In [78]:
berlin_districts["features"][0]['properties']["neighbourhood"]

'Blankenfelde/NiederschÃ¶nhausen'

In [69]:
berlin_districts["features"][0]["geometry"]

{'type': 'MultiPolygon',
 'coordinates': [[[[13.411909, 52.614871],
    [13.411826, 52.614838],
    [13.410914, 52.614604],
    [13.410822, 52.614578],
    [13.408516, 52.613961],
    [13.407365, 52.613653],
    [13.40751, 52.613593],
    [13.409923, 52.611614],
    [13.410344, 52.611269],
    [13.410476, 52.611161],
    [13.411191, 52.610709],
    [13.412582, 52.609868],
    [13.413236, 52.609557],
    [13.413555, 52.609405],
    [13.413611, 52.609223],
    [13.415022, 52.605169],
    [13.415149, 52.604873],
    [13.415831, 52.603062],
    [13.415854, 52.602967],
    [13.415843, 52.602537],
    [13.415996, 52.602017],
    [13.41626, 52.601544],
    [13.41659, 52.600657],
    [13.416776, 52.600064],
    [13.416871, 52.599751],
    [13.417183, 52.599564],
    [13.417927, 52.598957],
    [13.419275, 52.597546],
    [13.419679, 52.597683],
    [13.420086, 52.597456],
    [13.420236, 52.597373],
    [13.420521, 52.597214],
    [13.420771, 52.597076],
    [13.421442, 52.596774],
    [13.423

In [79]:
df_imported = pd.DataFrame(berlin_districts["features"][0]["geometry"])
df_imported.head(50)

,type,coordinates
0,MultiPolygon,"[[[13.411909, 52.614871], [13.411826, 52.61483..."


In [80]:
def query4sqr (query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=50"
    headers =  {
    "Accept": "application/json",
    "Authorization": f"{client_key}" # tengo que validar si poner la llave en la funcion es factible.
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        new_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, long]}}} for i in response]
    return new_list

In [81]:
bar = query4sqr("bar",[52.53211713308012, 13.370612229900534],3000)

In [82]:
bar

[{'name': 'Z - Bar Kulturverein...wechsel',
  'lat': 52.529415,
  'long': 13.395773,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [41.39804, 2.154868]}}},
 {'name': 'Böse Buben Bar',
  'lat': 52.522364,
  'long': 13.384082,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [41.39804, 2.154868]}}},
 {'name': 'Meine Bar ICI',
  'lat': 52.527313,
  'long': 13.397592,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [41.39804, 2.154868]}}},
 {'name': 'Basalt',
  'lat': 52.550713,
  'long': 13.360863,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [41.39804, 2.154868]}}},
 {'name': 'The Forsberg',
  'lat': 52.544445,
  'long': 13.371816,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [41.39804, 2.154868]}}},
 {'name': 'Kowski',
  'lat': 52.52786,
  'long': 13.341634,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [41.39804, 2.154868]}}},
 {'name': 'Lass uns Freunde bleiben',
  'lat': 52.533082,
  'long': 13.407